# Elicit Basics

This example will walk you through the basics of how to:
- Create an "Extractor", a controller which handles the extraction.
- Add required labelling functions and schemas to the extractor.
- Run the extraction process.
- Launch the user interface to begin annotating the extractions.

We will be using the existing Keyword Extractor labelling function as an example.

We begin by importing the requirements.

## Importing Requirements

In [1]:
# Import Extractor class and launch UI function.
from elicit import Extractor, launch_ui
# Import the Keyword Match Labelling Function.
from elicit.generic_labelling_functions import KeywordMatchLF, NLILabellingFunction
# Import Pathlib, for better path handling.
from pathlib import Path
# Import OS so we know where the notebook is!
import os

current_path = os.path.abspath('')

# get current directory
current_dir = Path(current_path)

docs = list((current_dir / "basic_example_docs").glob("*.txt"))

print("Current directory:", current_dir)
print("Documents:", [d.name for d in docs])

/home/dev/miniconda3/envs/elicit/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/dev/miniconda3/envs/elicit/lib/python3.9/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


Current directory: /home/dev/Turing/elicit/examples
Documents: ['doc_1.txt', 'doc_2.txt']


## Creating Extractor

Lets first create an Extractor object, pointing at the DB file we want to create.

In [2]:
# delete db if already exists (just for testing purposes)
(current_dir / "test_db.sqlite").unlink(missing_ok=True)

extractor = Extractor(db_path=current_dir / "test_db.sqlite")

Connected to Extraction Database: /home/dev/Turing/elicit/examples/test_db.sqlite


## Registering Schemas

Next, we will add the required schemas. These are the configuration files the labelling functions will use to extract the data.

In this case, we require:
- A categories schema
- A keywords schema

Categories is always required. It tells the system what categories each variable has, or whether it is numerical/raw. More details on this in the documentation.

Keywords are a dictionary of variable category to keyword list. Each category of a variable will have some user-defined set of keywords.

These schemas can either be a Path to a yaml file, or a dictionary.

In [3]:

categories = {"cat_or_dog": ["cat", "dog"]}
keywords = {"cat_or_dog": {"cat": ["meow", "hiss"], "dog": ["woof", "bark"]}}
questions = {"cat_or_dog": ["are you a cat?", "are you a dog?"]}

extractor.register_schema(schema=categories,
                            schema_name="categories")
extractor.register_schema(schema=keywords,
                            schema_name="keywords")
extractor.register_schema(schema=questions,
                            schema_name="questions")

Registered schema: categories
Registered schema: keywords
Registered schema: questions


## Registering Labelling Functions

Next, we register the labelling function. In this case, we have just imported the pre-defined Keyword Extractor labelling function. In a future tutorial, we will create our own labelling functions.

In [4]:
extractor.register_labelling_function(KeywordMatchLF)
extractor.register_labelling_function(NLILabellingFunction)

Registered labelling function: Keyword Match
Registered labelling function: Q&A → NLI Transformer


## Running the extractor

We can now run the extraction process, we pass a list of Paths pointing to each document. Currently PDFs and TXTs are supported.

In [5]:
extractor.run(docs)

Running LF: Keyword Match
Loading models and stuff...


Extracting variable: cat_or_dog: 100%|██████████| 2/2 [00:00<00:00,  2.44it/s]


Running LF: Q&A → NLI Transformer
Loading models and stuff...


Extracting variable: cat_or_dog:   0%|          | 0/2 [00:00<?, ?it/s]

[{'question': 'are you a cat?', 'context': 'The cat goes meow.'}, 1, {'question': 'are you a dog?', 'context': 'The cat goes meow.'}, 1]


ValueError: 1 argument needs to be of type (SquadExample, dict)

## Running the user interface

Finally, we can launch the user interface to begin annotating the extractions, pointing either to a database path, or simply passing in the extractor object.

In [ ]:
#launch_ui(extractor=extractor, output=True)